In [1]:
# %% 单元格1：导入所有需要的工具
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, Dropdown, Checkbox
from matplotlib.animation import FuncAnimation
from IPython.display import HTML, display
import sympy as sp  # 用于显示数学公式

# 设置中文字体（可选）
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

print("✅ 工具箱加载完毕！")

✅ 工具箱加载完毕！


In [2]:
# %% 单元格2：波动方程求解器（不用完全理解，直接复制）
def solve_wave_simple(c=1.0, L=10, T=5, nx=100, nt=200, bc_type='dirichlet'):
    """
    一维波动方程：u_tt = c^2 * u_xx
    参数：
        c - 波速
        L - 区域长度
        T - 总时间
        nx - 空间网格数
        nt - 时间步数
        bc_type - 边界条件：'dirichlet' 或 'neumann'
    """
    dx = L / (nx - 1)
    dt = T / (nt - 1)
    r = (c * dt / dx) ** 2  # CFL数
    
    # 网格
    x = np.linspace(0, L, nx)
    t = np.linspace(0, T, nt)
    u = np.zeros((nt, nx))
    
    # 初始条件：一个高斯波包
    x0 = L / 2
    sigma = L / 10
    u[0, :] = np.exp(-((x - x0) ** 2) / (2 * sigma ** 2))
    
    # 第一行的时间导数假设为0（静止释放）
    if nt > 1:
        u[1, :] = u[0, :]
    
    # 时间推进（简单的显式格式）
    for n in range(1, nt - 1):
        for i in range(1, nx - 1):
            u[n + 1, i] = (2 * (1 - r) * u[n, i] + 
                          r * (u[n, i + 1] + u[n, i - 1]) - 
                          u[n - 1, i])
        
        # 边界条件
        if bc_type == 'dirichlet':  # u=0
            u[n + 1, 0] = 0
            u[n + 1, -1] = 0
        elif bc_type == 'neumann':  # du/dx=0
            u[n + 1, 0] = u[n + 1, 1]
            u[n + 1, -1] = u[n + 1, -2]
    
    return x, t, u

print("✅ 波动方程求解器就绪！")

✅ 波动方程求解器就绪！


In [3]:
# %% 单元格3：创建交互式控件（最神奇的部分！）
@interact(
    wave_speed=FloatSlider(value=1.0, min=0.1, max=3.0, step=0.1, 
                          description='波速 c:', continuous_update=False),
    domain_length=FloatSlider(value=10, min=5, max=20, step=1,
                            description='区域长度 L:', continuous_update=False),
    boundary_type=Dropdown(options=['dirichlet', 'neumann'], 
                          value='dirichlet', description='边界条件:'),
    show_animation=Checkbox(value=True, description='显示动画'),
    time_step=FloatSlider(value=2.5, min=0, max=5, step=0.1,
                         description='当前时间 t:')
)
def update_wave_visualization(wave_speed=1.0, domain_length=10, 
                             boundary_type='dirichlet', 
                             show_animation=True, time_step=2.5):
    
    # 创建画布
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # 计算数值解
    x, t, u = solve_wave_simple(c=wave_speed, L=domain_length, 
                               bc_type=boundary_type)
    
    # 左图：当前时刻波形
    ax = axes[0]
    # 找到最接近选定时间的索引
    idx = np.argmin(np.abs(t - time_step))
    
    ax.plot(x, u[idx, :], 'b-', linewidth=3, label=f't = {t[idx]:.2f}s')
    ax.fill_between(x, 0, u[idx, :], alpha=0.3)
    ax.set_xlabel('位置 x', fontsize=12)
    ax.set_ylabel('振幅 u(x,t)', fontsize=12)
    ax.set_title(f'波形图 (c={wave_speed}, {boundary_type}边界)', fontsize=14)
    ax.grid(True, alpha=0.3)
    ax.legend()
    ax.set_ylim([-1.2, 1.2])
    
    # 右图：时空演化图
    ax = axes[1]
    im = ax.imshow(u, extent=[0, domain_length, 0, 5], 
                   aspect='auto', cmap='RdBu_r', origin='lower',
                   vmin=-1, vmax=1)
    ax.set_xlabel('位置 x', fontsize=12)
    ax.set_ylabel('时间 t', fontsize=12)
    ax.set_title('u(x,t) 时空演化', fontsize=14)
    
    # 添加当前时间的红线
    ax.axhline(y=time_step, color='red', linestyle='--', linewidth=2, 
               label=f'当前时间 t={time_step}')
    ax.legend()
    
    plt.colorbar(im, ax=ax, label='振幅')
    plt.tight_layout()
    plt.show()
    
    # 显示动画（如果需要）
    if show_animation:
        print("📽️ 正在生成动画...（请稍候）")
        
        # 创建动画
        fig_anim, ax_anim = plt.subplots(figsize=(8, 4))
        line, = ax_anim.plot([], [], 'b-', linewidth=3)
        
        ax_anim.set_xlim(0, domain_length)
        ax_anim.set_ylim(-1.5, 1.5)
        ax_anim.set_xlabel('位置 x')
        ax_anim.set_ylabel('振幅 u')
        ax_anim.set_title('波动传播动画')
        ax_anim.grid(True)
        
        # 动画更新函数
        def animate(frame):
            line.set_data(x, u[frame * 2, :])  # 每隔2帧取一帧加速
            ax_anim.set_title(f'波动传播动画 (t = {t[frame*2]:.2f}s)')
            return line,
        
        # 创建动画对象
        anim = FuncAnimation(fig_anim, animate, frames=len(t)//2, 
                           interval=50, blit=True)
        
        plt.close(fig_anim)  # 关闭图形，避免重复显示
        
        # 在Notebook中显示动画
        display(HTML(anim.to_jshtml()))
        
        print("✅ 动画生成完成！可以拖动上方滑块观察不同时刻的波形。")
    
    # 显示一些物理信息
    print("\n📊 物理信息：")
    print(f"  网格大小: Δx = {x[1]-x[0]:.3f}, Δt = {t[1]-t[0]:.3f}")
    print(f"  CFL数 (稳定性参数): {(wave_speed * (t[1]-t[0]) / (x[1]-x[0]))**2:.3f}")
    print(f"  能量最大值: {np.max(np.abs(u)):.3f}")

interactive(children=(FloatSlider(value=1.0, continuous_update=False, description='波速 c:', max=3.0, min=0.1), …